# AR classification demo

In [ ]:
%%capture
import grpc 
import numpy as np
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials
import keras
from keras.datasets import mnist
from keras import backend as K
import os
from pathlib import Path

In [ ]:
mapping = {0:'sitting',1: 'standing',2: 'walking',3: 'running', 4: 'biking'}
def decode(index):
    return mapping[index]

#### Setup channel

In [ ]:
channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="activity_recognition")

# Normal case

Use test sample from the training domain (the model was trained on data collected from sensor on hip position).

#### Read data sample

In [31]:
path = Path(os.getcwd()).parent
Xtest = np.load(f'{path}/data/SHL5hip_300_X_test.npy')[:,0]
Ytest = np.load(f'{path}/data/SHL5hip_300_Y_test.npy')[:,0]
yval = np.array(Ytest)[1]
val = np.array(Xtest)[1].reshape(1,300,3,3).astype(np.double)
val = val.flatten()

#### Create tensor

In [32]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=300),hs.TensorShapeProto.Dim(size=3),hs.TensorShapeProto.Dim(size=3)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=val)

#### Send request to model servable

In [33]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor})
# print(request)
result = stub.Predict(request)
result = np.array(result.outputs['y'].double_val).argmax()
print(f'Result activity:  {decode(result)}')
print(f'Actual activity:  {decode(yval.argmax())}')

Result activity:  walking
Actual activity:  walking


# Outlier case

Use sample from another data domain. Data collected from another sensor position (hand) in our case.

#### Read data sample

In [34]:
Xtest = np.load(f'{path}/data/SHL5hand_300_X_test.npy')[:,0]
Ytest = np.load(f'{path}/data/SHL5hand_300_Y_test.npy')[:,0]
yval2 = np.array(Ytest)[1]
val2 = np.array(Xtest)[1].reshape(1,300,3,3).astype(np.double)
val2 = val.flatten()*2

#### Create tensor

In [35]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=300),hs.TensorShapeProto.Dim(size=3),hs.TensorShapeProto.Dim(size=3)])
tensor2 = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=val2)

#### Send request to model servable

In [36]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor2})
result = stub.Predict(request)
result = np.array(result.outputs['y'].double_val).argmax()
print(f'Result activity:  {decode(result)}')
print(f'Actual activity:  {decode(yval2.argmax())}')

Result activity:  walking
Actual activity:  standing
